In [1]:
import location_utils

## load gv data

In [2]:
from glob import glob
import os
import pandas
all_frames = [os.path.basename(path)
              for path in glob('../EventRegistries/GunViolence/frames/*')
              if not 'Icon' in path
             ]
frames = all_frames
df = pandas.concat([pandas.read_pickle('../EventRegistries/GunViolence/frames/' + frame)
                    for frame in frames])


## mapping gun violence surface forms to uris
* query dbpedia to check for city or country within state
* use external module based on address

In [16]:
state_sf2dbpedia_uri = location_utils.get_state_sf2dbpedia_uri(df)

In [17]:
state_sf2dbpedia_uri['New York']

'http://dbpedia.org/resource/New_York'

In [4]:
len(df)

3469

In [5]:
incident2city_or_county_uris = dict()
counter = 0
for index, row in df.iterrows():
    state = row['state']
    if state in state_sf2dbpedia_uri:
        state_uri = state_sf2dbpedia_uri[state]
        results = location_utils.possible_db_uris_for_city_county(row['city_or_county'],
                                                                  state_uri)
        incident2city_or_county_uris[row['incident_uri']] = {
            'sparql' : results,
            'state_uri' : state_uri,
            'city_or_county': row['city_or_county']
        }
        counter += 1
    else:
        print(state)

District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia
District of Columbia


In [6]:
from collections import Counter

In [7]:
distr = Counter( len(value['sparql']) 
                 for value in incident2city_or_county_uris.values())

In [8]:
succes = 100 * (1806 / sum(distr.values()))
succes

52.5305410122164

In [9]:
distr

Counter({0: 387,
         1: 1806,
         2: 651,
         3: 242,
         4: 43,
         5: 79,
         6: 178,
         7: 37,
         8: 1,
         9: 5,
         12: 4,
         13: 1,
         14: 1,
         16: 1,
         18: 1,
         25: 1})

In [10]:
for incident_uri, info in incident2city_or_county_uris.items():
    if len(info['sparql']) == 2:
        print()
        print(incident_uri)
        print(info)
        print(df.loc[df.incident_uri == incident_uri])
        break


797824
{'sparql': {'http://dbpedia.org/resource/Rochester,_Ulster_County,_New_York', 'http://dbpedia.org/resource/Rochester,_New_York'}, 'state_uri': 'http://dbpedia.org/resource/New_York', 'city_or_county': 'Rochester'}
  incident_uri            date     state city_or_county        address  \
0       797824  March 19, 2017  New York      Rochester  Orange Street   

   num_killed  num_injured                                       incident_url  \
0           1            0  http://www.gunviolencearchive.org/incident/797824   

                                    incident_sources  \
0  {'http://web.archive.org/web/20170319232851/ht...   

                                        participants gvdb_annotation  
0  [{'Age Group': ' Adult 18+', 'Type': ' Victim'...              {}  


In [11]:
sum(distr.values())

3438

In [12]:
distr

Counter({0: 387,
         1: 1806,
         2: 651,
         3: 242,
         4: 43,
         5: 79,
         6: 178,
         7: 37,
         8: 1,
         9: 5,
         12: 4,
         13: 1,
         14: 1,
         16: 1,
         18: 1,
         25: 1})

In [13]:
len(set(df['incident_uri']))

3469

In [14]:
incident_uri_distr = Counter(df['incident_uri'])

In [15]:
df.loc[df.incident_uri == '783782']

,incident_uri,date,state,city_or_county,address,num_killed,num_injured,incident_url,incident_sources,participants,gvdb_annotation
15,783782,"February 26, 2017",Tennessee,Clifton,2373 Culps Bend Road,1,0,http://www.gunviolencearchive.org/incident/783782,{'http://web.archive.org/web/20170316050930/ht...,"[{'Age Group': ' Teen 12-17', 'Type': ' Victim...",{}
